# 大众点评评价情感分析~
先上结果：

| 糖水店的评论文本                             | 模型预测的情感评分 |
| :------------------------------------------- | :----------------- |
| '糖水味道不错，滑而不腻，赞一个，下次还会来' | 0.91               |
| '味道一般，没啥特点'                         | 0.52               |
| '排队老半天，环境很差，味道一般般'           | 0.05               |

模型的效果还可以的样子，yeah~接下来我们好好讲讲怎么做的哈，我们通过爬虫爬取了大众点评广州8家最热门糖水店的3W条评论信息以及评分作为训练数据，前面的分析我们得知*样本很不均衡*。接下来我们的整体思路就是：文本特征处理(分词、去停用词、TF-IDF)—机器学习建模—模型评价。

我们先不处理样本不均衡问题，直接建模后查看结果，接下来我们再按照两种方法处理样本不均衡，对比结果。

### 数据读入和探索

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import jieba
data = pd.read_csv('data.csv')
data.head()

,cus_id,comment_time,comment_star,cus_comment,kouwei,huanjing,fuwu,shopID,stars,year,month,weekday,hour,comment_len
0,迷糊泰迪,2018-09-20 06:48:00,sml-str40,南信 算是 广州 著名 甜品店吧 ，好几个 时间段 路过 ，都是 座无虚席。 看着 餐单 ...,非常好,好,好,518986.0,4.0,2018.0,9.0,3.0,6.0,184.0
1,稱霸幼稚園,2018-09-22 21:49:00,sml-str40,中午吃完了所谓的早茶 回去放下行李 休息了会 就来吃下午茶了[服务]两层楼 楼下只能收现金...,很好,很好,很好,518986.0,4.0,2018.0,9.0,5.0,21.0,266.0
2,爱吃的美美侠,2018-09-22 22:16:00,sml-str40,【VIP冲刺王者战队】【吃遍蓉城战队】【VIP有特权】五月份和好朋友毕业旅行来了广州。我们都...,很好,很好,很好,518986.0,4.0,2018.0,9.0,5.0,22.0,341.0
3,姜姜会吃胖,2018-09-19 06:36:00,sml-str40,都说来广州吃糖水就要来南信招牌姜撞奶，红豆双皮奶牛三星，云吞面一楼现金，二楼微信支付宝位置不...,非常好,很好,很好,518986.0,4.0,2018.0,9.0,2.0,6.0,197.0
4,forevercage,2018-08-24 17:58:00,sml-str50,一直很期待也最爱吃甜品，广州的甜品很丰富很多样，来之前就一直想着一定要过来吃到腻，今天总算实...,非常好,很好,很好,518986.0,5.0,2018.0,8.0,4.0,17.0,261.0


### 构建标签值

大众点评的评分分为1-5分，1-2为差评，4-5为好评，3为中评，因此我们把1-2记为0,4-5记为1,3为中评，对我们的情感分析作用不大，丢弃掉这部分数据，但是可以作为训练语料模型的语料。我们的情感评分可以转化为标签值为1的概率值，这样我们就把情感分析问题转为文本分类问题了。

In [2]:
#构建label值
def zhuanhuan(score):
    if score > 3:
        return 1
    elif score < 3:
        return 0
    else:
        return None
    
#特征值转换
data['target'] = data['stars'].map(lambda x:zhuanhuan(x))
data_model = data.dropna()

### 文本特征处理

中文文本特征处理，需要进行中文分词，jieba分词库简单好用。接下来需要过滤停用词，网上能够搜到现成的。最后就要进行文本转向量，有词库表示法、TF-IDF、word2vec等，这篇文章作了详细介绍，推荐一波 https://zhuanlan.zhihu.com/p/44917421

这里我们使用sklearn库的TF-IDF工具进行文本特征提取。

In [3]:
#切分测试集、训练集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_model['cus_comment'], data_model['target'], random_state=2, test_size=0.25)

In [4]:
#引入停用词
infile = open("stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

In [5]:
#中文分词
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df
 
x_train_fenci = fenci(x_train)
x_train_fenci[:5]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\bin\AppData\Local\Temp\jieba.cache
Loading model cost 0.828 seconds.
Prefix dict has been built succesfully.


2861     双皮奶 不错 ， 重点 推荐 。 牛 三星 也 不错 ， 各种 料 都 很 嫩 ， 但是 汤...
4790     漫步 上下 九 ， 走 着 走 着 就 来到 了 南信 ， 忍不住 尝 一下 它 的 双皮奶...
7273     # 双皮奶 #   非常 有 奶味 ， 和 外面 吃 的 完全 不 一样 ， 绝对 值得 一...
15199    好像 是 很 有名 的 甜品店 ， 在 门口 的 地方 满满 一面 墙 都 是 菜单 ， 天...
32068     传统 甜品店 ， 每次 西华路 吃完饭 都 会 来 叹 翻碗 甜品 ， 老城区 生活 真 唔 错
Name: cus_comment, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv.fit(x_train_fenci)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=3000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '--', '.', '..', '...', '......', '...................', './', '.一', '记者', '数', '年', '月', '日', '时', '分', '秒', '/', '//', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '://', '::', ';', '<', '=', '>', '>>', '?', '@'...3', '94', '95', '96', '97', '98', '99', '100', '01', '02', '03', '04', '05', '06', '07', '08', '09'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

### 机器学习建模

这里我们使用文本分类的经典算法朴素贝叶斯算法

In [7]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(tv.transform(fenci(x_train)), y_train)
classifier.score(tv.transform(fenci(x_test)), y_test)

0.9312315634218289

In [8]:
from sklearn.metrics import roc_auc_score, f1_score
y_pred = classifier.predict_proba(tv.transform(fenci(x_test)))[:,1]
roc_auc_score(y_test,y_pred)

0.8932292907834565

In [9]:
def ceshi(model,strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(model.predict_proba(tv.transform(strings_fenci))[:,1])

In [10]:
#从大众点评网找两条评论来测试一下
test1 = '很好吃，环境好，所有员工的态度都很好，上菜快，服务也很好，味道好吃，都是用蒸馏水煮的，推荐，超好吃' #5星好评
test2 = '糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。' #1星差评
print('好评实例的模型预测情感得分为{}\n差评实例的模型预测情感得分为{}'.format(ceshi(classifier,test1),ceshi(classifier,test2)))

好评实例的模型预测情感得分为0.8628561324354109
差评实例的模型预测情感得分为0.8240768974016524


可以看出，准确率和AUC值都非常不错的样子，但点评网上的实际测试中，5星好评模型预测出来了，1星差评缺预测错误。为什么呢？我们查看一下**混淆矩阵**

In [11]:
from sklearn.metrics import confusion_matrix
y_predict = classifier.predict(tv.transform(fenci(x_test)))
cm = confusion_matrix(y_test, y_predict)
cm

array([[  64,  369],
       [   4, 4987]], dtype=int64)

可以看出，负类的预测非常不准，433单准确预测为负类的只有15.7%，应该是由于数据不平衡导致的，模型的默认阈值为输出值的中位数。比如逻辑回归的输出范围为[0,1]，当某个样本的输出大于0.5就会被划分为正例，反之为反例。在数据的类别不平衡时，采用默认的分类阈值可能会导致输出全部为正例，产生虚假的高准确度，导致分类失败。处理样本不均衡问题的方法，首先可以选择调整阈值，使得模型对于较少的类别更为敏感，或者选择合适的评估标准，比如ROC或者F1，而不是准确度（accuracy）。另外一种方法就是通过采样（sampling）来调整数据的不平衡。其中欠采样抛弃了大部分正例数据，从而弱化了其影响，可能会造成偏差很大的模型，同时，数据总是宝贵的，抛弃数据是很奢侈的。另外一种是过采样，下面我们就使用过采样方法来调整。

### 过采样（单纯复制）

单纯的重复了反例，因此会过分强调已有的反例。如果其中部分点标记错误或者是噪音，那么错误也容易被成倍的放大。因此最大的风险就是对反例过拟合。

In [12]:
data['target'].value_counts()

1.0    19915
0.0     1779
Name: target, dtype: int64

In [13]:
#把0类样本复制10次
index_tmp = y_train==0
y_tmp = y_train[index_tmp]
x_tmp = x_train[index_tmp]
x_train2 = pd.concat([x_train,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp])
y_train2 = pd.concat([y_train,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp])

In [14]:
clf2 = MultinomialNB()
clf2.fit(tv.transform(fenci(x_train2)), y_train2)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
y_pred2 = clf2.predict_proba(tv.transform(fenci(x_test)))[:,1]
roc_auc_score(y_test,y_pred2)

0.9040843957923339

In [16]:
y_predict2 = clf2.predict(tv.transform(fenci(x_test)))
cm = confusion_matrix(y_test, y_predict2)
cm

array([[ 325,  108],
       [ 631, 4360]], dtype=int64)

可以看出，即使是简单粗暴的复制样本来处理样本不平衡问题，负样本的识别率大幅上升了，变为77%，慢慢的幸福感呀~我们自己写两句评语来看看

In [17]:
ceshi(clf2,'排队人太多，环境不好，口味一般')

0.3100641465700192

可以看出把0类别的识别出来了，太棒了~

### 过采样（SMOTE算法）

SMOTE（Synthetic minoritye over-sampling technique,SMOTE），是在局部区域通过K-近邻生成了新的反例。相较于简单的过采样，SMOTE降低了过拟合风险，但同时运算开销加大

对SMOTE感兴趣的同学可以看下这篇文章https://www.jianshu.com/p/ecbc924860af

In [18]:
from imblearn.over_sampling import SMOTE
oversampler=SMOTE(random_state=0)
x_train_vec = tv.transform(fenci(x_train))
x_resampled, y_resampled = oversampler.fit_sample(x_train_vec, y_train)

In [19]:
y_train.value_counts()

1.0    14923
0.0     1346
Name: target, dtype: int64

In [20]:
pd.Series(y_resampled).value_counts()

0.0    14923
1.0    14923
dtype: int64

我们经过插值，把0类数据也丰富为14923个数据了，这时候正负样本的比例为1:1，接下来我们用平衡后的数据进行训练，效果如何呢，好期待啊~

In [21]:
clf3 = MultinomialNB()
clf3.fit(x_resampled, y_resampled)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
y_pred3 = clf3.predict_proba(tv.transform(fenci(x_test)))[:,1]
roc_auc_score(y_test,y_pred3)

0.9036572990736674

In [23]:
y_predict3 = clf3.predict(tv.transform(fenci(x_test)))
cm = confusion_matrix(y_test, y_predict3)
cm

array([[ 328,  105],
       [ 593, 4398]], dtype=int64)

In [24]:
#到网上找一条差评来测试一下效果
test3 = '糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。'
ceshi(clf3,test3)

0.3003167657573628

可以看出，使用SMOTE插值与简单的数据复制比起来，0类的召回率有提高了

### 模型评估测试

接下来我们把3W条数据都拿来训练，数据量变多了，模型效果应该会更好

In [25]:
#词向量训练
tv2 = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv2.fit(fenci(data_model['cus_comment']))

#SMOTE插值
X_tmp = tv2.transform(fenci(data_model['cus_comment']))
y_tmp = data_model['target']
sm = SMOTE(random_state=0)
X,y = sm.fit_sample(X_tmp, y_tmp)

clf = MultinomialNB()
clf.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
def fenxi(strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(clf.predict_proba(tv2.transform(strings_fenci))[:,1])

In [27]:
#到网上找一条差评来测试一下
fenxi('糯米外皮不绵滑，豆沙馅粗躁，没有香甜味。12元一碗不值。')

0.28941535322192086

只用到了简单的机器学习，就做出了不错的情感分析效果，知识的力量真是强大呀，666~
### 后续优化方向

- 使用更复杂的机器学习模型如神经网络、支持向量机等
- 模型的调参
- 行业词库的构建
- 增加数据量
- 优化情感分析的算法
- 增加标签提取等
- 项目部署到服务器上，更好地分享和测试模型的效果